In [1]:
folder_path = "../data/jss/imrg/ds1/testbed_1_s"
best_path = "../data/jss/imrg/ds1/best/testbed1_small.csv"
output_file = "results/meal.csv"

In [28]:
import os, csv, time, math
import numpy as np
from mealpy import TS, SA, ACOR, GA, Problem, PermutationVar


class JSSP(Problem):
    def __init__(self, bounds, minmax, name="JSSP", processing_times=None, **kwargs):
        self.name = name
        self.processing_times = processing_times
        super().__init__(bounds, minmax, **kwargs)

    def obj_func(self, solution):
        solution = self.decode_solution(solution)["per"]
        num_jobs = len(solution)
        num_machines = len(self.processing_times[0])
        
        # Initialize completion times matrix
        completion_times = [[0] * num_machines for _ in range(num_jobs)]
        total_completion_time = 0

        for job_index in range(num_jobs):
            job = solution[job_index]

            for machine in range(num_machines):
                if job_index == 0:
                    if machine == 0:
                        completion_times[job_index][machine] = self.processing_times[job][machine]
                    else:
                        completion_times[job_index][machine] = completion_times[job_index][machine-1] + self.processing_times[job][machine]
                else:
                    if machine == 0:
                        completion_times[job_index][machine] = completion_times[job_index-1][machine] + self.processing_times[job][machine]
                    else:
                        completion_times[job_index][machine] = self.processing_times[job][machine] + max(completion_times[job_index][machine-1], completion_times[job_index-1][machine])

                if machine == num_machines - 1:
                    total_completion_time += completion_times[job_index][machine]

        return total_completion_time
    
    def amend_position(self, solution):
        unique_jobs = np.unique(solution)
        if len(unique_jobs) == len(solution):
            # The solution is already a valid permutation
            return solution
        else:
            # If it's not, we reconstruct a valid permutation
            # This scenario might be rare depending on how the solution is generated or modified
            n = len(solution)
            valid_permutation = np.random.permutation(n)
            return valid_permutation.tolist()

In [3]:
def read_file_input(filepath):
    with open(filepath) as f:
        lines = f.readlines()
        sizes = [int(nr) for nr in lines[0].split()]
        m, n = sizes[0], sizes[1]
        arr = np.zeros((n, m))

        for i in range(1, len(lines)):
            arr[i-1, :] = [int(nr) for nr in lines[i].split()]

    return arr

In [4]:
def read_all_files_in_folder(folder_path):
    # List to hold file contents
    file_contents = {}

    # Check if the provided path is a directory
    if not os.path.isdir(folder_path):
        raise ValueError(f"The provided path '{folder_path}' is not a valid directory.")

    # Iterate over all files in the directory
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)

        # Check if it's a file
        if os.path.isfile(file_path):
            try:
                # Open and read the file
                with open(file_path, 'r') as file:
                    content = file.read()
                    file_contents[filename] = content
            except Exception as e:
                print(f"Error reading file {filename}: {e}")
    
    return file_contents

In [5]:
def process_files(file_data):
    results = []
    total_time = 0
    for filename, content in file_data.items():
        data = read_file_input(f"{folder_path}/{filename}")
        problem_jssp = JSSP(bounds=PermutationVar(valid_set=[i for i in range(data.shape[0])], name="per"), processing_times=data, minmax="min", log_to="")
        model = SA.OriginalSA(epoch=1400, temp_init=100, cooling_rate=0.995)
        
        # FIXME: seems like tabu search is bugged
        # model = TS.OriginalTS(epoch=1000, pop_size=50, tabu_size=5, neighbour_size=20, perturbation_scale=0.05)
        # model = GA.BaseGA(epoch=1000, pop_size=50, pc=0.9, pm=0.05, selection="tournament")
        # model = ACOR.OriginalACOR(epoch=1000, pop_size=50, sample_count = 25, intent_factor = 0.5, zeta = 1.0)

        start_time = time.time()
        model.solve(problem=problem_jssp)
        end_time = time.time()
        duration = end_time - start_time

        print(f"Processing file: {filename} took {duration:.4f}s")

        results.append({"filename": filename, "cost": model.g_best.target.fitness, "time": duration})
        total_time += duration

    print(f"Total time spent solving: {total_time:.4f}s")
    return results

In [6]:
def write_results_to_csv(results, output_file):
    with open(output_file, 'w', newline='') as csvfile:
        fieldnames = ['filename', 'cost', 'time']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()
        for result in results:
            writer.writerow(result)

In [7]:
file_data = read_all_files_in_folder(folder_path)

In [8]:
results = process_files(file_data)

Processing file: t1s_0238.txt took 0.6820s
Processing file: t1s_0216.txt took 0.6415s
Processing file: t1s_0312.txt took 0.5089s
Processing file: t1s_0281.txt took 0.4861s
Processing file: t1s_0113.txt took 0.4722s
Processing file: t1s_0082.txt took 0.3373s
Processing file: t1s_0192.txt took 0.4013s
Processing file: t1s_0266.txt took 0.3469s
Processing file: t1s_0021.txt took 0.4094s
Processing file: t1s_0018.txt took 0.3737s
Processing file: t1s_0162.txt took 0.3841s
Processing file: t1s_0147.txt took 0.3788s
Processing file: t1s_0183.txt took 0.4594s
Processing file: t1s_0346.txt took 0.5709s
Processing file: t1s_0062.txt took 0.4031s
Processing file: t1s_0027.txt took 0.3937s
Processing file: t1s_0200.txt took 0.4930s
Processing file: t1s_0111.txt took 0.4096s
Processing file: t1s_0213.txt took 0.6110s
Processing file: t1s_0034.txt took 0.3540s
Processing file: t1s_0295.txt took 0.4047s
Processing file: t1s_0057.txt took 0.3629s
Processing file: t1s_0328.txt took 0.5155s
Processing 

In [9]:
results.sort(key=lambda x: x["filename"])
write_results_to_csv(results, output_file)

# Results

## SA

| temp_init | cooling_rate | epoch | Time |
|-----------|--------------|------------|------|
| 1000 | 0.9995 | 41500 | 4783s |
| 1000 | 0.9995 | 27500 | 3165s |
| 100 | 0.9995 | 13800 | 1519s |
| 100 | 0.999 | 6900 | 748s |
| 100 | 0.995 | 1400 | 132s |


# Tuning Params

In [30]:
from skopt.space import Real, Integer
from skopt import gp_minimize

In [50]:
def compute_overall_score_SA(params):
    # epoch, temp_init, cooling_rate = params
    cooling_rate = params[0]
    total_score = 0

    for filename, content in file_data.items():
        data = read_file_input(f"{folder_path}/{filename}")
        problem_jssp = JSSP(bounds=PermutationVar(valid_set=[i for i in range(data.shape[0])], name="per"), processing_times=data, minmax="min", log_to="")
        model = SA.OriginalSA(epoch=1400, temp_init=100, cooling_rate=cooling_rate)
        model.solve(problem=problem_jssp)

        total_score += model.g_best.target.fitness

    return total_score

In [51]:
# Define the search space for the parameters
space = [
    # Integer(1000, 15000, name='epoch'),
    # Real(50, 500, name='temp_init'),
    Real(0.95, 0.9995, name='cooling_rate')
]

# Perform Bayesian optimization
res = gp_minimize(compute_overall_score_SA, space, n_calls=100, random_state=0)

0.9793458086021384
2532622.0
0.9917911545547604
2520622.0
0.9924683080723266
2540505.0
0.9919389610698143
2531689.0
0.9808664029909057
2532680.0
0.9690268945109887
2532113.0
0.9647279630239514
2529200.0
0.9528072923772134
2543438.0
0.9634964865817156
2536788.0
0.9736444233074069
2529970.0
0.9913422301822873
2529876.0
0.9916723575001553
2523083.0
0.9916023553256337
2523612.0
0.9655785741827382
2527051.0
0.9662194927246258
2541983.0
0.9651952876975529
2543630.0
0.9915950782606657
2535132.0
0.9976949843633858
2528549.0
0.9995
2535930.0
0.9916588590781203
2525435.0
0.9916597300798639
2542739.0
0.9915303233059177
2525820.0
0.9916078074180464
2536860.0
0.99163650506425
2523959.0
0.9910935092836725
2525621.0
0.9890342847701454
2536327.0
0.9913848405122606
2540986.0
0.9916785882064448
2529502.0
0.991692308051166
2532066.0
0.9916906701943395
2531290.0
0.9930111381386757
2531255.0
0.9916905085409875
2534227.0
0.9916767661837214
2527157.0
0.9916994429888827
2533579.0
0.9916819949623503
2538466.0


In [53]:
# Output the best parameters found
print("Best parameters found:")
print(f"k: {res.x[0]}")

print("Best score achieved:")
print(res.fun)

Best parameters found:
k: 0.9994994149984398
Best score achieved:
2517532.0


In [58]:
def compute_overall_score_GA(params):
    pc, pm = params
    total_score = 0

    print(f"Params: {params}")

    for filename, content in file_data.items():
        data = read_file_input(f"{folder_path}/{filename}")
        problem_jssp = JSSP(bounds=PermutationVar(valid_set=[i for i in range(data.shape[0])], name="per"), processing_times=data, minmax="min", log_to="")
        model = GA.BaseGA(epoch=100, pop_size=20, pc=pc, pm=pm, selection="tournament")
        model.solve(problem=problem_jssp)

        total_score += model.g_best.target.fitness

    print(f"Total score: {total_score}")
    return total_score

In [60]:
# Define the search space for the parameters
space = [
    Real(0.8, 0.99, name='pc'),
    Real(0.1, 0.99, name='pm')
]

# Perform Bayesian optimization
res = gp_minimize(compute_overall_score_GA, space, n_calls=100, random_state=0)

Params: [0.9126404774627536, 0.8513965162371057]
Total score: 2405733.0
Params: [0.9630096673483238, 0.8540540475178718]
Total score: 2407721.0
Params: [0.9184771023893348, 0.442099719490503]
Total score: 2406368.0
Params: [0.8565315752434498, 0.15047454981252445]
Total score: 2401319.0
Params: [0.8518046959702216, 0.5251219544160015]
Total score: 2404907.0
Params: [0.9543120584673437, 0.5271796834138012]
Total score: 2406782.0
Params: [0.8746291112591577, 0.8441100995482662]
Total score: 2406365.0
Params: [0.864105270479281, 0.6768729661255656]
Total score: 2405252.0
Params: [0.8699658925697041, 0.9518680914682115]
Total score: 2408223.0
Params: [0.8266666482784026, 0.8743776599390085]
Total score: 2405869.0
Params: [0.8, 0.1]
Total score: 2402044.0
Params: [0.8941088567739597, 0.1]
Total score: 2405654.0


/home/marin/.local/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [0.8, 0.1] before, using random point [0.9044211906511079, 0.19922535102351924]
  warnings.warn(


Params: [0.9044211906511079, 0.19922535102351924]
Total score: 2403079.0


/home/marin/.local/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [0.8, 0.1] before, using random point [0.8924575993680728, 0.2905611265781124]
  warnings.warn(


Params: [0.8924575993680728, 0.2905611265781124]
Total score: 2403474.0


...

Params: [0.8334599651349204, 0.4340336369109087]
Total score: 2405246.0


/home/marin/.local/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [0.8, 0.1] before, using random point [0.8231954592354698, 0.1452876893291141]
  warnings.warn(


Params: [0.8231954592354698, 0.1452876893291141]
Total score: 2400801.0


/home/marin/.local/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [0.8, 0.1] before, using random point [0.8983759065697413, 0.8043749309044962]
  warnings.warn(


Params: [0.8983759065697413, 0.8043749309044962]
Total score: 2407065.0


In [61]:
# Output the best parameters found
print("Best parameters found:")
print(f"pc: {res.x[0]}")
print(f"pm: {res.x[1]}")

print("Best score achieved:")
print(res.fun)

Best parameters found:
pc: 0.8231954592354698
pm: 0.1452876893291141
Best score achieved:
2400801.0


In [78]:
def compute_overall_score_ACO(params):
    sample_count, intent_factor, zeta = params
    # intent_factor, zeta = params
    print(f"Params: {params}")
    total_score = 0

    for filename, content in file_data.items():
        data = read_file_input(f"{folder_path}/{filename}")
        problem_jssp = JSSP(bounds=PermutationVar(valid_set=[i for i in range(data.shape[0])], name="per"), processing_times=data, minmax="min", log_to="")
        model = ACOR.OriginalACOR(epoch=50, pop_size=20, sample_count=int(sample_count), intent_factor=intent_factor, zeta=zeta)
        model.solve(problem=problem_jssp)
        
        total_score += model.g_best.target.fitness

    print(f"Total score: {total_score}")
    return total_score

In [80]:
# Define the search space for the parameters
space = [
    Integer(10, 25, name='sample_count'),
    Real(0.1, 0.99, name='intent_factor'),
    Real(0.1, 4.99, name='zeta')
]

# Perform Bayesian optimization
res = gp_minimize(compute_overall_score_ACO, space, n_calls=20, random_state=0)

Params: [19, 0.8513965162371057, 4.295354070175282]
Total score: 2420611.0
Params: [23, 0.6549716901395155, 1.979626548661303]
Total score: 2414572.0
Params: [14, 0.15047454981252445, 1.433289280496754]
Total score: 2419300.0
Params: [17, 0.8228301686101891, 2.447088372914031]
Total score: 2420526.0
Params: [16, 0.8441100995482662, 1.749867224440443]
Total score: 2419678.0
Params: [20, 0.4277349704580877, 4.780488727280398]
Total score: 2418014.0
Params: [12, 0.8743776599390085, 2.415943341388445]
Total score: 2428239.0
Params: [22, 0.5632249568005724, 3.4197209022817168]
Total score: 2415532.0
Params: [21, 0.6179976149468455, 2.7277550920056632]
Total score: 2417828.0
Params: [21, 0.19425777039713504, 2.4159060506051557]
Total score: 2417297.0
Params: [25, 0.1, 4.99]
Total score: 2416230.0
Params: [23, 0.6513338095319289, 2.06873690924435]
Total score: 2414084.0
Params: [23, 0.7373529925815903, 4.99]
Total score: 2415515.0
Params: [24, 0.4656078310643821, 0.7764789825141976]
Total sco

/home/marin/.local/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [25, 0.1, 0.1] before, using random point [21, 0.9144740848013515, 4.496750688844827]
  warnings.warn(


Params: [21, 0.9144740848013515, 4.496750688844827]
Total score: 2418655.0


/home/marin/.local/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [25, 0.1, 0.1] before, using random point [17, 0.8645498099261213, 4.972280061334667]
  warnings.warn(


Params: [17, 0.8645498099261213, 4.972280061334667]
Total score: 2420593.0


/home/marin/.local/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [25, 0.1, 0.1] before, using random point [11, 0.8841527323879416, 3.156693087260733]
  warnings.warn(


Params: [11, 0.8841527323879416, 3.156693087260733]
Total score: 2428664.0


/home/marin/.local/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [25, 0.1, 0.1] before, using random point [12, 0.5522576792796648, 0.5068680810305788]
  warnings.warn(


Params: [12, 0.5522576792796648, 0.5068680810305788]
Total score: 2417715.0


In [82]:
# Output the best parameters found
print("Best parameters found:")
print(f"sample_count: {res.x[0]}")
print(f"intent_factor: {res.x[1]}")
print(f"zeta: {res.x[2]}")

print("Best score achieved:")
print(res.fun)

Best parameters found:
sample_count: 25
intent_factor: 0.1
zeta: 0.1
Best score achieved:
2393327.0
